In [8]:
import pandas as pd
import numpy as np
import requests

## Data Gathering

In [9]:
# url of our data
basic_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
title_aka_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
title_rating_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [10]:
# create a list for url and names
url_list = [basic_url, title_aka_url, title_rating_url]
url_names = ['title.basics.tsv.gz', 'title.akas.tsv.gz', 'title.ratings.tsv.gz']

In [11]:
# use request to download data Programmatically from the url
responses = []
for url in url_list:
    response = requests.get(url)
    responses.append(response.content)
n = 0
for names in url_names:
    with open(names, mode='wb') as file: 
        file.write(responses[n])
    n += 1

In [12]:
title_basics = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False)

In [13]:
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [14]:
title_aka = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory = False)

In [15]:
title_aka.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [16]:
title_aka.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35843078 entries, 0 to 35843077
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


In [17]:
title_rating = pd.read_csv('title.ratings.tsv.gz', sep='\t', low_memory=False)

In [18]:
title_rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1969
1,tt0000002,5.8,263
2,tt0000003,6.5,1815
3,tt0000004,5.6,178
4,tt0000005,6.2,2612


In [19]:
title_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308743 entries, 0 to 1308742
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1308743 non-null  object 
 1   averageRating  1308743 non-null  float64
 2   numVotes       1308743 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.0+ MB


## Data Cleaning

### Title Basics column:

1. Replace "\N" with np.nan
2. Eliminate movies that are null for runtimeMinutes
3. Eliminate movies that are null for genre
4. keep only titleType==Movie
5. keep startYear 2000-2022
6. Eliminate movies that include "Documentary" in genre 
7. Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [20]:
# Create a copy
title_basics_clean = title_basics.copy()

**1. Replace "\N" with np.nan**

In [21]:
title_basics_clean.replace({'\\N':np.nan}, inplace=True)

**2. Eliminate movies that are null for runtimeMinutes**

In [22]:
title_basics_clean.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1330687
endYear           9731354
runtimeMinutes    6937611
genres             443082
dtype: int64

In [23]:
missing_index = title_basics_clean[title_basics_clean.runtimeMinutes.isnull()].index

In [24]:
# drop rows
title_basics_clean.drop(missing_index, inplace=True)

In [25]:
# test
title_basics_clean.runtimeMinutes.isna().sum()

0

**3. Eliminate movies that are null for genre**

In [26]:
missing_genre_index = title_basics_clean[title_basics_clean.genres.isnull()].index

In [27]:
# drop rows
title_basics_clean.drop(missing_genre_index, inplace=True)

In [28]:
# test
title_basics_clean.genres.isna().sum()

0

**4. keep only titleType==Movie**

In [29]:
# select movie type
title_basics_clean = title_basics_clean[title_basics_clean.titleType == 'movie']

In [30]:
# test
title_basics_clean.titleType.unique()

array(['movie'], dtype=object)

**5. keep startYear 2000-2022**

In [31]:
title_basics_clean = title_basics_clean[title_basics_clean.startYear.isin(['2000', '2001', '2002'])]

In [32]:
# test
title_basics_clean.startYear.unique()

array(['2001', '2002', '2000'], dtype=object)

**6. Eliminate movies that include "Documentary" in genre**

In [33]:
# movie index that includes Documentary
is_doc_index = title_basics_clean[title_basics_clean.genres.str.contains('documentary',case=False)].index

In [34]:
# drop rows
title_basics_clean.drop(is_doc_index, inplace=True)

In [35]:
# Test
title_basics_clean.genres.str.contains('documentary',case=False).unique()

array([False])

**7. Keep only US movies**

In [36]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = title_basics_clean['tconst'].isin(title_aka['titleId'])

In [37]:
title_basics_clean = title_basics_clean[keepers]
title_basics_clean.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
100076,tt0102362,movie,Istota,Istota,0,2000,NaN,80,"Drama,Romance"
110364,tt0112912,movie,Dune 7,Dune 7,0,2002,NaN,97,Adventure
110476,tt0113026,movie,The Fantasticks,The Fantasticks,0,2000,NaN,86,"Musical,Romance"


### Title AKAs Column:

1. Replace "\N" with np.nan
2. keep only US movies.

In [38]:
# create a copy
title_aka_clean = title_aka.copy()

**1. Replace "\N" with np.nan**

In [39]:
title_aka_clean.replace({'\\N':np.nan}, inplace=True)

**2. keep only US movies.**

In [40]:
title_aka_clean = title_aka_clean[title_aka_clean.region == 'US']

In [41]:
# test
title_aka_clean.region.unique()

array(['US'], dtype=object)

### Title Ratings Column:

1. Replace "\N" with np.nan (if any)
2. Keep only US movies (Use AKAs table)

In [42]:
# create a copy
title_rating_clean = title_rating.copy()

**1. Replace "\N" with np.nan (if any)**

In [43]:
title_rating_clean.replace({'\\N':np.nan}, inplace=True)

**2. Keep only US movies (Use AKAs table)**

In [44]:
# Filter the rating table down to only include the US by using the filter akas dataframe
keepers = title_rating_clean['tconst'].isin(title_aka['titleId'])

In [45]:
title_rating_clean = title_rating_clean[keepers]
title_rating_clean.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1969
1,tt0000002,5.8,263
2,tt0000003,6.5,1815
3,tt0000004,5.6,178
4,tt0000005,6.2,2612


In [46]:
# making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 

# Confirm folder created
os.listdir("Data/")

['title_aka_clean.csv.gz',
 'title_basics_clean.csv.gz',
 'title_rating_clean.csv.gz']

In [47]:
## Save current dataframe to file.
title_basics_clean.to_csv("Data/title_basics_clean.csv.gz",compression='gzip',index=False)

In [48]:
title_aka_clean.to_csv("Data/title_aka_clean.csv.gz",compression='gzip',index=False)

In [49]:
title_rating_clean.to_csv("Data/title_rating_clean.csv.gz",compression='gzip',index=False)

In [50]:
# Open saved file and preview again
title_basics = pd.read_csv("Data/title_basics_clean.csv.gz", low_memory = False)
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
2,tt0102362,movie,Istota,Istota,0,2000,NaN,80,"Drama,Romance"
3,tt0112912,movie,Dune 7,Dune 7,0,2002,NaN,97,Adventure
4,tt0113026,movie,The Fantasticks,The Fantasticks,0,2000,NaN,86,"Musical,Romance"


In [51]:
# Open saved file and preview again
title_rating = pd.read_csv("Data/title_rating_clean.csv.gz", low_memory = False)
title_rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1969
1,tt0000002,5.8,263
2,tt0000003,6.5,1815
3,tt0000004,5.6,178
4,tt0000005,6.2,2612
